# Исследование надёжности заёмщиков

Исследование для кредитного отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.describe()




,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
df.isna().sum()
total_income_median = df['total_income'].median()
days_employed_mean = df['days_employed'].mean()
df['days_employed'] = df['days_employed'].fillna(days_employed_mean)
df['total_income'] = df['total_income'].fillna(total_income_median)
df['dob_years'].value_counts()
df['gender'].value_counts()

df.head(30)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Замена типа данных

In [4]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()
df['days_employed'] = df['days_employed'].abs()
df['education'] = df['education'].str.lower()

df.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Обработка дубликатов

In [5]:

df['children'].value_counts()
df['children'] = df['children'].replace(-1, 1)
children_median = df.loc[df.loc[:, 'children'] != 20]['children'].median()
df['children'] = df['children'].replace(20, children_median)
df['children'].median()


print('количество детей у заемщиков', df['children'].value_counts())



количество детей у заемщиков 0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


In [11]:
df.duplicated().sum() #нашли количество дубликатов 71
#df.drop_duplicates()
df.drop_duplicates().reset_index(drop = True)
#удалили кол - во дубликатов



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод** Проверили на дубликаты столбецы children и family_status. В первом обнаружены ошибки с значением -1 и 20. Можно допустить что в первом значении случайно поставили минус, а во втором тоже опечатка, должно быть либо 0 либо 2. Так же применял метод duplicated но он какой либо нужной информации не дал. Методом реплейс меняем значение с -1 на 1. Мадианой обрабатываем значение 20 


In [6]:
from pymystem3 import Mystem 
m = Mystem()


In [7]:


df['lemma'] = df['purpose'].apply(m.lemmatize)

df['lemma'].value_counts()

[автомобиль, \n]                                          973
[свадьба, \n]                                             797
[на,  , проведение,  , свадьба, \n]                       777
[сыграть,  , свадьба, \n]                                 774
[операция,  , с,  , недвижимость, \n]                     676
[покупка,  , коммерческий,  , недвижимость, \n]           664
[операция,  , с,  , жилье, \n]                            653
[покупка,  , жилье,  , для,  , сдача, \n]                 653
[операция,  , с,  , коммерческий,  , недвижимость, \n]    651
[покупка,  , жилье, \n]                                   647
[жилье, \n]                                               647
[покупка,  , жилье,  , для,  , семья, \n]                 641
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        634
[операция,  , со,  , свой,  , недвижимость, \n]           630
[строительство,  , жилой,  , недвижимость, \n]            626
[покупка

In [8]:
def credit_purposes(lemma):
    if 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'свадьба' in lemma:
        return 'свадьба'
    elif 'образование' in lemma:
        return 'образование'
    elif 'жилье' in lemma or 'недвижимость' in lemma:
        return 'недвижимость'
    else:
        return 'другое'

df['main_purposes'] = df['lemma'].apply(credit_purposes)

display(df.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma,main_purposes
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость


In [9]:
# c помощью метода cut выделил 3 возможных уровня дохода. Это низкий, средний, высокий.
income_cut = pd.qcut(df['total_income'], 3)
income_cut.value_counts()

                              



(170468.0, 2265604.0]      7175
(123082.667, 170468.0]     7175
(20666.999, 123082.667]    7175
Name: total_income, dtype: int64

In [10]:
def find_income_group(income):
    if income < 123082:
        return('low')
    if 123082 <= income <= 170468:
        return('medium')
    if 20666 <= income <= 2265604:
        return('high')
    
#Сгруппировали доход по 3 категориям методом if

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [11]:
data_pivot = df.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count')
data_pivot["процент"] =(data_pivot[1]/ (data_pivot[0] + data_pivot[1])) * 100
print(data_pivot.head(10))


debt            0       1   процент
children                           
0         13154.0  1071.0  7.528998
1          4420.0   445.0  9.146968
2          1861.0   194.0  9.440389
3           303.0    27.0  8.181818
4            37.0     4.0  9.756098
5             9.0     NaN       NaN


**Вывод**Зависимость имеется. Сделал таблицу с 3 колонками при помощи функции data_pivot.  Где столбец 0 это количество не просроченных кредитов, столбец 1 с количеством не сдавших в срок и колонка процент для общей доли.  Мы видим что люди у которых нет детей закрывают кредит в срок. Люди у которых имеет 4 детей, чаще всего не успевают сдать кредит вовремя.
    

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [12]:
data_pivot = df.pivot_table(index='family_status', columns='debt', values='purpose', aggfunc='count')
data_pivot["процент"] =(data_pivot[1]/ (data_pivot[0] + data_pivot[1])) * 100
print(data_pivot.head(10))

debt                       0    1   процент
family_status                              
Не женат / не замужем   2539  274  9.740491
в разводе               1110   85  7.112971
вдовец / вдова           897   63  6.562500
гражданский брак        3789  388  9.288963
женат / замужем        11449  931  7.520194


**Вывод**
Зависимость имеется. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [13]:
#data_pivot = df.pivot_table(index=['total_income'], columns='debt', values='purpose', aggfunc='count')
#print(data_pivot.head(30))
debtors_income = df[['total_income', 'debt']]
debtors_income = debtors_income.loc[debtors_income['debt'] == 1]
debtors_income['income_group'] = debtors_income['total_income'].apply(find_income_group)
debtors_income[['income_group', 'debt']].groupby('income_group').sum().sort_values('debt', ascending = False)

,debt
income_group,
medium,615
low,583
high,543


**Вывод** Поделили клиентов на 3 категории по уровню дохода. В результате подсчета должников видим, что больше всего таковых в категории дохода medium но  количество должников в категориях отличается немного.

Поэтому зависимости между уровнем дохода и возвратом кредита в срок нет.

- Как разные цели кредита влияют на его возврат в срок?

In [14]:
data_pivot = df.pivot_table(index='main_purposes', columns='debt', values='purpose', aggfunc='count')
data_pivot["процент"] =(data_pivot[1]/ (data_pivot[0] + data_pivot[1])) * 100
print(data_pivot.head(10))

debt               0    1   процент
main_purposes                      
автомобиль      3912  403  9.339513
недвижимость   10058  782  7.214022
образование     3652  370  9.199403
свадьба         2162  186  7.921635


**Вывод**Здесь нам понадобилось разбиение целей кредита по категориям.  Среди 4 наших категорий чаще всего платят в срок с целью кредита недвижимость.Чаще просрочка у тех кому нужен кредит на авто и образование. Кредиты на свадьбу просрачивают реже  примерно как и в случае недвижимости.

**Общий вывод**

1.Семейный статус влияет на вероятность платежей по кредиту в срок люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. 2.Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату. 3.Зависимости между уровнем дохода и возвратом кредита в срок нет. 